In [23]:
# Importing modules
###############################
import datetime
import numpy as np
import pandas as pd
from IPython.display import clear_output # To make prints look cleaner

def Dataset(url1, url2, local1, final_loc):
    """
    A function to import, transform, and return the data needed for 
    the visualizations.
    
    Args:
        url1(string): A url to a dataset.
        url2(string): A url to a dataset.
        local1(string): A path to a local file.
        final_loc(string): Where to save the file.
    
    Returns: 
        final_df(DataFrame): DataFrame of data.
    """
    
    # Keeping track of progress
    print('Importing data.')
    
    # Loading the various datasets
    confirmed   = get_data(url1)
    deaths      = get_data(url2)
    population  = get_data(local1)
    
    # Making sure that both timeseries are updated by comparing the latest dates
    assert [*confirmed.columns][-1] == [*deaths.columns][-1], 'Timeseries do not match.'
    
    # Clearing output to keep things clean
    clear_output(wait=True)
    
    # Progress check
    print('Data has been imported.')
    print('Fixing FIPS values.')
    
    # Fixing/adding FIPS columns
    confirmed   = fix_fips(confirmed)
    deaths      = fix_fips(deaths)
    population  = fix_fips(population)
    
    # Setting index of population
    population = population.set_index('FIPS')
    
    clear_output(wait=True)
    
    print('FIPS values fixed.')
    print('Generating dictionaries.')
    
    # Generating dicitonaries
    list_dict, state_dict = get_dicts(confirmed, population)
    
    clear_output(wait=True)
    
    print('Dictionaries generated.')
    print('Making a base DataFrame.')
    
    final_df = combine_data(confirmed, deaths, population, list_dict, state_dict)
    
    clear_output(wait=True)
    print('DataFrame created.')
    print('Adding more columns.')
    
    final_df['New Cases'] = new_column(column = 'Confirmed', 
                                       kind = 'average', 
                                       data = final_df,
                                       data2 = population,
                                       dict1 = list_dict,
                                       days=1)
    
    final_df['New Deaths'] = new_column(column = 'Deaths', 
                                        kind = 'average',  
                                        data = final_df,
                                        data2 = population,
                                        dict1 = list_dict,
                                        days = 1)
    
    final_df['Cases, 7DMA'] = new_column(column = 'Confirmed', 
                                         kind = 'average', 
                                         data = final_df,
                                         data2 = population,
                                         dict1 = list_dict, 
                                         days = 7)
    
    final_df['Deaths, 7DMA'] = new_column(column = 'Deaths',
                                          kind = 'average', 
                                          data = final_df,
                                          data2 = population,
                                          dict1 = list_dict,
                                          days = 7)
    
    final_df['New Cases Per 100k, 7DMA'] = new_column(column = 'Cases, 7DMA',
                                                      kind = 'PER100K', 
                                                      data = final_df,
                                                      data2 = population,
                                                      dict1 = list_dict
                                                     )

    final_df['Deaths Per 100k'] = new_column(column = 'Deaths',
                                                       kind = 'PER100K', 
                                                       data = final_df,
                                                       data2 = population,
                                                       dict1 = list_dict
                                                      )    
    
    # Removing decimals
    final_df = final_df.astype('int')
    
    clear_output(wait=True)
    print("New columns added.")
    print('Saving file.')
    
    final_df.to_csv(final_loc, index = False)
    
    clear_output(wait=True)
    print('File saved.')
    print('Have fun with visualizations!')
    
    return final_df, list_dict
    
def get_data(location):
    """
    Import data from the given location.
    
    Args: 
        location(string): Where to import the data from.
                
    Returns:
        df(DataFrame): DataFrame of the data at the given location.
    """
    
    # Importing data to a pandas DataFrame
    df = pd.read_csv(location)
    
    # Returning dataframe
    return df

def fix_fips(df):
    """
    Formatting FIPS values.
    
    Returns:
        df(DataFrame): Updated dataframe.
    """
    

    
    # Making sure that the FIPS values are strings, and zero-padded.
    df['FIPS'] = df['FIPS'].fillna(0).astype('int32').astype('str').str.zfill(5)
    
    
    # Returning the new dataframe
    return df

def get_dicts(df1, df2):
    """
    Generates various dicts, to make things easier and more readable.
    
    Returns:
        list_dict(dict): Dictionary of lists.
    """
    
    # Making a list of all FIPS values in the population set (States and counties)
    pop_fips = [*df2.index]
    
    # Making a list of county fips values, these are between '01001' and '56045' (included)
    index_01001 = int(*df1[df1['FIPS']=="01001"].index)
    index_56045 = int(*df1[df1['FIPS']=="56045"].index)
    county_fips = list(df1.loc[index_01001:index_56045,'FIPS'])
    
    # Making a list of state fips, this is the difference between the two prior lists
    # Sorting because sets are unsorted
    state_fips = sorted(list(set(pop_fips) - set(county_fips)))
    
    # Adding a fips value for the US total '00000' to the pop_fips
    all_fips = ['00000']
    all_fips.extend(pop_fips)
    all_fips = sorted(all_fips)
    
    # Making a list of dates found in the timeseries
    dates = [*df1.iloc[:,11:].columns]
    
    # Adding the lists to a dictionary
    list_dict = {}
    
    list_dict['all_fips'] = all_fips
    list_dict['state_fips'] = state_fips
    list_dict['county_fips'] = county_fips
    list_dict['pop_fips'] = pop_fips
    list_dict['dates'] = dates
    
    # Making a dict of state names
    state_dict = {fips: df2.loc[fips,'STNAME'] for fips in state_fips}

    # Returning the dictionaries
    return list_dict, state_dict

def combine_data(df1,df2,df3,dict1,dict2):
    """
    Making a new dataframe to use for vizualizations. 
    
    Returns:
        df(DataFrame): A Multiindex DataFrame with FIPS and date as indexes. 
    """
    # Creating dictionaries for counties, state totals, and overall totals
    # Where the keys are tuples of the fips and the date
  
    usa_total = {('00000', date): [np.sum(df1[date].values),np.sum(df2[date].values)] for date in dict1['dates']}

    
    # This part is basically just for me to quickly see how many new cases and deaths in the last day
    snapshot = usa_total[('00000',dict1['dates'][-2])]
    snap = usa_total[('00000',dict1['dates'][-1])]
    s = np.array(snap)-np.array(snapshot)
    print(f"There were {s[0]} new cases and {s[1]} new deaths on {dict1['dates'][-1]}.")
    
    counties_dict = {(fips, date): [df1.loc[int(*df1[df1['FIPS'] == fips].index),date],\
                                    df2.loc[int(*df2[df2['FIPS'] == fips].index),date]]\
                     for fips in dict1['county_fips'] for date in dict1['dates'] } 
    
    clear_output(wait=True)
    print('County dict generated.')
    print('Generating state dict.')
    
    states = {(fips, date): [np.sum(df1[df1['Province_State'] == dict2[fips]][date]),\
                             np.sum(df2[df2['Province_State'] == dict2[fips]][date])]\
              for fips in dict1['state_fips'] for date in dict1['dates']}
    
    clear_output(wait=True)
    print('State dict generated.')
    print('Generating Multiindex.')
    
    # Transforming to DataFrames, and transposing to have the wanted structure
    df = pd.DataFrame(counties_dict).transpose()
    usa_df = pd.DataFrame(usa_total).transpose()
    states_df = pd.DataFrame(states).transpose()
    
    # Naming columns and indexes
    columns = ['Confirmed','Deaths']
    index = ['FIPS','Date']
    
    # Stacking the DataFrames on top of each other
    final_df = pd.concat([df,usa_df,states_df],axis=0)
    
    # Naming columns and indexes
    final_df.columns = columns
    final_df.index.names = index
    
    # Popping the index out to make the dates into datetime and re-setting index
    final_df = final_df.reset_index()
    final_df['Date']=pd.to_datetime(final_df['Date'],format='%m/%d/%y')
    final_df = final_df.set_index(index)
    
    # Sorting the dataframe, which is why the dates had to be datetime!
    final_df = final_df.sort_index()
    
    # Returning the dataframe that will be added to
    return final_df

def new_column(column, kind, data, data2, dict1, days = 1):
    """
    Generates a list of new values to be added as a new column.
    
    Args:
        column(string): Column to do calculations on.
        data(df): dataframe that holds the data
        data2(df): population data
        dict1(dictionary): dict with different lists
        kind(string): kwarg to decide what type of calculation.
            'average': average between dates in the timeseries. Number of days given by days.
            'capita': per 100k population.
        days(int): Number of days 
    
    Returns:
        lst(list): list of values to be the new column.
    """
    lst = []
    # Checking if we're doing a difference calculation
    if kind == 'average':
        
        # Looping over all fips values
        for fips in dict1['all_fips']:
            
            # Taking the difference between days, and then dividing by that number of days
            temp = round(data.loc[fips, column].diff(periods=days).fillna(0)/days,0)
            
            # For each FIPS value add to the list
            lst.extend(temp)
            
        # Returning the list
        return lst
        
    # Checking if we're doing a per 100k calculation
    elif kind == 'PER100K':
        
        # First finding the overall population (scaled)
        total = 0
        
        for fips in dict1['state_fips']:
            total = total + data2.loc[fips,kind]
            
        # A special case for '00000'
        temp = round(data.loc['00000',column] / total, 0)
        lst.extend(temp)
        
        # For the rest of the FIPS
        for fips in dict1['pop_fips']:
            
            temp = round(data.loc[fips,column] / data2.loc[fips,kind],0)
            lst.extend(temp)
            
        return lst

In [24]:
# Timeseries of confirmed cases in the US
url1 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"

# Timeseries of confirmed deathes in the US
url2 = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"

# The modified population estimate
local1 = 'data/population.csv'

final_loc = 'data/us_covid.csv'
us_covid, list_dict = Dataset(url1, url2, local1, final_loc) 

File saved.
Have fun with visualizations!


In [ ]:
# Taking the columns I want from the population estimates

# Estimated data of the US population in states and counties
# Downloaded from: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html#par_textimage_70769902
pop = pd.read_csv("data/co-est2019-alldata.csv", engine = 'python')

# Fixing the FIPS to the wanted format
pop['FIPS'] = [str(row[1].STATE).zfill(2)+str(row[1].COUNTY).zfill(3)\
                          for row in pop.iterrows()]

pop['PER100K'] = pop['POPESTIMATE2019'] / 100000
# Taking the columns of interest and saving the file
pop = pop[['FIPS','STNAME','CTYNAME','POPESTIMATE2019','PER100K']]
pop.to_csv('data/population.csv', index = False)

In [25]:
# USA
us_covid.loc['00000'].tail()

,Confirmed,Deaths,New Cases,New Deaths,"Cases, 7DMA","Deaths, 7DMA","New Cases Per 100k, 7DMA",Deaths Per 100k
Date,,,,,,,,
2020-07-15,3498902,137415,67328,949,63458,731,19,42
2020-07-16,3576157,138358,77255,943,65459,724,20,42
2020-07-17,3647715,139266,71558,908,65997,738,20,42
2020-07-18,3711413,140119,63698,853,66498,763,20,43
2020-07-19,3773260,140534,61847,415,66903,761,20,43


In [22]:
# Florida
us_covid.loc['12000'].tail()

,Confirmed,Deaths,New Cases,New Deaths,"Cases, 7DMA","Deaths, 7DMA","New Cases Per 100k, 7DMA",Deaths Per 100k
Date,,,,,,,,
2020-07-15,301810,4521,10181,112,11147,90,52,21
2020-07-16,315775,4677,13965,156,11865,95,55,22
2020-07-17,327241,4805,11466,128,11870,100,55,22
2020-07-18,337569,4895,10328,90,11865,100,55,23
2020-07-19,350047,4982,12478,87,11462,106,53,23


In [16]:
# California
us_covid.loc['06000'].tail()

,Confirmed,Deaths,New Cases,New Deaths,"Cases, 7DMA","Deaths, 7DMA","New Cases Per 100k, 7DMA",Deaths Per 100k
Date,,,,,,,,
2020-07-15,354885,7375,8674,125,8904,94,23,19
2020-07-16,364706,7489,9821,114,8889,90,22,19
2020-07-17,373259,7603,8553,114,8911,93,23,19
2020-07-18,380745,7702,7486,99,8855,96,22,19
2020-07-19,386906,7715,6161,13,8909,95,23,20


In [17]:
# San Francisco
us_covid.loc['06075'].tail()

,Confirmed,Deaths,New Cases,New Deaths,"Cases, 7DMA","Deaths, 7DMA","New Cases Per 100k, 7DMA",Deaths Per 100k
Date,,,,,,,,
2020-07-15,4696,50,56,0,89,0,10,6
2020-07-16,4860,51,164,1,102,0,12,6
2020-07-17,4975,52,115,1,94,0,11,6
2020-07-18,5116,52,141,0,99,0,11,6
2020-07-19,5202,52,86,0,97,0,11,6


In [18]:
# Texas
us_covid.loc['48000'].tail()

,Confirmed,Deaths,New Cases,New Deaths,"Cases, 7DMA","Deaths, 7DMA","New Cases Per 100k, 7DMA",Deaths Per 100k
Date,,,,,,,,
2020-07-15,290892,3506,11315,144,9423,90,32,12
2020-07-16,305854,3657,14962,151,9902,93,34,13
2020-07-17,317768,3820,11914,163,10264,107,35,13
2020-07-18,327106,3939,9338,119,10384,114,36,14
2020-07-19,335257,4012,8151,73,10356,114,36,14


# Covid exploring

I'm just going to play around with some datasets, make some visualizations, and maybe do some machine learning after that.

In [ ]:
# Importing datasets
###############################

# Population data found here: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html#par_textimage_70769902
population = pd.read_csv("data/co-est2019-alldata.csv", engine = 'python')

confirmed = pd.read_csv(
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv", engine='python')
deaths = pd.read_csv(
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

In [ ]:
# Making sure that both files are updated
assert [*confirmed.columns][-1] == [*deaths.columns][-1]

In [ ]:
# Making a FIPS list from the population estimates data, this includes state FIPS.

fips_list = [str(row[1].STATE).zfill(2)+str(row[1].COUNTY).zfill(3) for row in population.iterrows()]
population['FIPS'] = fips_list

# Filling NA's with 0, and setting FIPS to a zeropadded string.
confirmed['FIPS'] = confirmed['FIPS'].fillna(0).astype('int32').astype('str').str.zfill(5)
deaths['FIPS'] = deaths['FIPS'].fillna(0).astype('int32').astype('str').str.zfill(5)

# Making a second fips list to see where we need to make some special rules.
# In this dataset I only care about FIPS ranging from '01001' to '56045'.
# These are the FIPS values for the counties in the US.
index_01001 = int(*confirmed[confirmed['FIPS']=="01001"].index)
index_56045 = int(*confirmed[confirmed['FIPS']=="56045"].index)
county_fips = list(confirmed.loc[index_01001:index_56045,'FIPS'])

states_fips = sorted(list(set(fips_list) - set(county_fips)))
print(len(states_fips))

# Appending a FIPS for the total US
fips_list.append('00000')
fips_list = sorted(fips_list)

By looking at the length of the exceptions list we can see that there are 51 FIPS in the population data that are not in the John Hopkins data, plus the one that I added for the total of the US. These are one for each state, and then one for District of Columbia. That means I have 52 exceptions to deal with when making the dataframe. For the states I'll make a dictionary with state names, and then go over the JHU dataset and take sums of the rows with that state name, this will include unassigned and prison cases. The same for DC. For USA I'll take a sum of all rows, this includes territories as well which I feel should be counted to the US total.

In [ ]:
# Creating a list of all the dates in the dataset

dates = [*confirmed.iloc[:,11:].columns]

# Creating a state names dictionary
states_dict = {fips: population.iloc[int(*population[population['FIPS']==fips].index),5] for fips in states_fips}

# Just a sanity check to see what dates are included
print(dates[0],dates[-1])

In [ ]:
# Creating dictionaries for counties, state totals, and overall totals
# Where the keys are tuples of the fips and the date
counties_dict = {(fips, date): [confirmed.loc[int(*confirmed[confirmed['FIPS'] == fips].index),date], deaths.loc[int(*deaths[deaths['FIPS'] == fips].index),date]] for fips in county_fips for date in dates } 


states = {(fips, date): [np.sum(confirmed[confirmed['Province_State'] == states_dict[fips]][date]),np.sum(deaths[deaths['Province_State'] == states_dict[fips]][date])] for fips in states_fips for date in dates}


usa_total = {('00000', date): [np.sum(confirmed[date].values),np.sum(deaths[date].values)] for date in dates}

In [ ]:
usa_total = {('00000', date): [np.sum(confirmed[date].values),np.sum(deaths[date].values)] for date in dates}
usa_df = pd.DataFrame(usa_total).transpose()


In [ ]:
usa_df['New']= usa_df[0].diff()

In [ ]:
usa_df.tail()

In [ ]:

# Transforming to DataFrames, and transposing to have the wanted structure
df = pd.DataFrame(counties_dict).transpose()
usa_df = pd.DataFrame(usa_total).transpose()
states_df = pd.DataFrame(states).transpose()

# Naming columns and indexes
columns = ['Confirmed','Deaths']
index = ['FIPS','Date']

# Stacking the DataFrames on top of each other
final_df = pd.concat([df,usa_df,states_df],axis=0)


final_df.columns = columns
final_df.index.names = index
final_df = final_df.reset_index()
final_df['Date']=pd.to_datetime(final_df['Date'],format='%m/%d/%y')
final_df = final_df.set_index(index)

final_df = final_df.sort_index()

print(final_df.tail(), final_df.head(),final_df.shape)


In [ ]:
lst = []

for fips in fips_list:
    temp = final_df.loc[fips,'Confirmed'].diff()
    lst.extend(temp)
    
final_df['New Cases'] = lst

lst = []

for fips in fips_list:
    temp = final_df.loc[fips,'Deaths'].diff()
    lst.extend(temp)
    
final_df['New Deaths'] = lst

lst = []

for fips in fips_list:
    temp = final_df.loc[fips,'Confirmed'].diff(7)/7
    lst.extend(temp)
    
final_df['Cases, 7DMA'] = lst

lst = []

for fips in fips_list:
    temp = final_df.loc[fips,'Deaths'].diff(7)/7
    lst.extend(temp)
    
final_df['Deaths, 7DMA'] = lst
final_df = final_df.fillna(0)
final_df = final_df.astype('int')

In [ ]:
# Checking San Francisco data

final_df.swaplevel(0,1,axis=0).loc[('2020-07-07','06075')]

In [ ]:
final_df.loc['00000'].tail()

In [ ]:
final_df.to_excel('wtf.xlsx')

In [ ]:
testing_df = final_df.loc['00000':'01001',:]
testing_df.loc[('00000',)].tail()

In [ ]:
testing_df = testing_df.reset_index()
testing_df['Date']=pd.to_datetime(testing_df['Date'],format='%m/%d/%y')
testing_df = testing_df.set_index(['FIPS','Date'])

In [ ]:
testing_df.loc['00000',:].tail()